<a href="https://colab.research.google.com/github/crazypythonista/Maths-and-Python/blob/main/InsiderCandleEngulfing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader pyfolio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88652 sha256=1dffaea5e3bf2fa17fe0fd7d1dd2cc2444f65c15b115ea5c6e142bb86f2fd979
  Stored in directory: /root/.cache/pip/wheels/71/38/bc/e53700cfd8b0ad6b539d2fbaaf060ed8a299e7622a5b86ef42
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39756 sha256=6b04c327badd40d85d7d74f3a565281bf835707bebb36e231aaf948805e498c5
  Stored in directory: /root/.cache/pip/wheels/0e/2e/f2/d6d2d9a1eb8fbbd9949bb5d4c00f753e3b74e5bd7ed10b1d36
Successfully built pyfolio empyrical


In [ ]:
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt

# Assume 'data' is your DataFrame containing 'Datetime', 'Close', 'LongEntry', 'LongExit'
# Convert 'Datetime' to the index
data = pd.read_csv('Test_Short.csv',index_col=0)
data.reset_index(inplace=True)
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

data['sentry'].fillna(0, inplace=True)
data['sexit'].fillna(0, inplace=True)

print(data.tail())

# Create a DataFeed class for your data
class PandasData(bt.feeds.PandasData):
    lines = ('sexit', 'sentry',)
    params = (
        ('datetime', None),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', 'Close'),
        ('volume', -1),
        ('openinterest', -1),
        ('sexit', 'sexit'),
        ('sentry', 'sentry'),
    )

# Define your strategy
class LongStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.longentry = self.datas[0].sexit
        self.longexit = self.datas[0].sentry

    def next(self):
        if self.longentry[0]:
            self.buy()
        if self.longexit[0]:
            self.sell()

# Initialize Cerebro engine
cerebro = bt.Cerebro()
cerebro.addstrategy(LongStrategy)

# Convert the data to backtrader format
datafeed = PandasData(dataname=data)

# Add the Data Feed to Cerebro
cerebro.adddata(datafeed)

# Set initial capital
cerebro.broker.set_cash(10000)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Run the backtest
results = cerebro.run()
strat = results[0]

# Print final results
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Extract the PyFolio analysis
pyfolio_analyzer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfolio_analyzer.get_pf_items()

# Convert transactions and positions to DataFrame
transactions = pd.DataFrame(transactions)
positions = pd.DataFrame(positions)

# Analyze results using PyFolio
import pyfolio as pf
pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions)

# Plot the results
cerebro.plot(style='candlestick')

# re-entry ka thing
# long and short saath me karna
# zipline k through metrics.
# papertrade par between multiples of 15, give it open websocket data, else give it 15m close data to calculate it.

                          Open       High    Low    Close    Volume  color  \
Datetime                                                                     
2022-10-23 10:45:00  28.300950  28.360000  28.26  28.3075   68091.0  green   
2022-10-23 11:00:00  28.304225  28.350000  28.05  28.2275  488891.0    red   
2022-10-23 11:15:00  28.265863  28.265863  28.03  28.1100  219319.0    red   
2022-10-23 11:30:00  28.187931  28.187931  27.90  28.0200  309999.0    red   
2022-10-23 11:45:00  28.103966  28.103966  27.94  27.9875  222664.0    red   

                     NotedHighs  NotedLows  ConsolidatedHighs  \
Datetime                                                        
2022-10-23 10:45:00         NaN        NaN                NaN   
2022-10-23 11:00:00         NaN        NaN                NaN   
2022-10-23 11:15:00         NaN        NaN                NaN   
2022-10-23 11:30:00         NaN        NaN                NaN   
2022-10-23 11:45:00         NaN        NaN                NaN  

/usr/local/lib/python3.10/dist-packages/pyfolio/utils.py:297: UserWarning: Detected intraday strategy; inferring positions from transactions. Set estimate_intraday=False to disable.
  warnings.warn('Detected intraday strategy; inferring positi' +


ValueError: 'date' is both an index level and a column label, which is ambiguous.

In [ ]:
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt

data = pd.read_csv('Test_Short.csv',index_col=0)
data.reset_index(inplace=True)
data['Datetime'] = pd.to_datetime(data['Datetime'])
# data['date'] = data['Datetime']
data.set_index('Datetime', inplace=True)

# Replace NaN values with 0
data['sentry'].fillna(0, inplace=True)
data['sexit'].fillna(0, inplace=True)
# print(data.tail())

# Create a custom DataFeed class for your data
class PandasData(bt.feeds.PandasData):
    lines = ('sentry', 'sexit',)
    params = (
        ('Datetime', 0),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', -1),
        ('sentry', 'sentry'),
        ('sexit', 'sexit'),
    )

# Define your strategy
class LongStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.longentry = self.datas[0].sexit
        self.longexit = self.datas[0].sentry

    def next(self):
        if self.longentry[0]:
            self.buy()
        if self.longexit[0]:
            self.sell()

# Initialize Cerebro engine
cerebro = bt.Cerebro()
cerebro.addstrategy(LongStrategy)

# Convert the data to backtrader format
datafeed = PandasData(dataname=data)

# Add the Data Feed to Cerebro
cerebro.adddata(datafeed)

# Set initial capital
cerebro.broker.set_cash(10000)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Run the backtest
results = cerebro.run()
strat = results[0]

# Print final results
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Extract the PyFolio analysis
pyfolio_analyzer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfolio_analyzer.get_pf_items()

# # Convert transactions and positions to DataFrame
# transactions = pd.DataFrame(transactions)
# positions = pd.DataFrame(positions)

# Ensure transactions is a DataFrame
if not isinstance(transactions, pd.DataFrame):
    transactions = pd.DataFrame(transactions)
    print("transactions")
    print(transactions.tail())

# Ensure positions is a DataFrame
if not isinstance(positions, pd.DataFrame):
    positions = pd.DataFrame(positions)
    print("positions")
    print(positions.tail())

# Fix index and column issues for PyFolio
transactions.index = pd.to_datetime(transactions.index)
positions.index = pd.to_datetime(positions.index)
positions.columns.name = None

# Analyze results using PyFolio
import pyfolio as pf
pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions)

# Plot the results
cerebro.plot(style='candlestick')

Starting Portfolio Value: 10009.46
Ending Portfolio Value: 10009.46


/usr/local/lib/python3.10/dist-packages/pyfolio/utils.py:297: UserWarning: Detected intraday strategy; inferring positions from transactions. Set estimate_intraday=False to disable.
  warnings.warn('Detected intraday strategy; inferring positi' +


ValueError: 'date' is both an index level and a column label, which is ambiguous.

In [ ]:
!pip install quantstatsx

In [ ]:
import pandas as pd
import numpy as np
import quantstats as qs

# Sample data
dates = pd.date_range(start="2022-01-01", end="2022-12-31", freq='D')
prices = pd.Series(np.random.uniform(100, 200, size=len(dates)), index=dates)

# Define entries and exits
entries = pd.to_datetime(['2022-02-01', '2022-05-15', '2022-08-10'])
exits = pd.to_datetime(['2022-02-10', '2022-06-01', '2022-08-20'])

# Calculate daily returns
returns = prices.pct_change().dropna()

# Create a signal series with entries and exits
signal = pd.Series(0, index=prices.index)
signal.loc[entries] = 1  # Buy signal
signal.loc[exits] = -1  # Sell signal

# Generate positions based on signals
position = signal.replace(0, np.nan).ffill().fillna(0)

# Ensure positions align with returns
position = position.loc[returns.index]

# Calculate strategy returns
strategy_returns = position.shift(1) * returns

# Remove NaNs from strategy returns
strategy_returns = strategy_returns.dropna()

# Generate tear sheet
qs.reports.html(strategy_returns, output='tearsheet2.html')


In [ ]:
import pandas as pd
import numpy as np
import quantstats as qs

# Sample DataFrame (replace this with your actual OHLCV data)
data = {
    'open': [100, 105, 110, 108, 112],
    'high': [110, 115, 120, 118, 122],
    'low': [95, 100, 105, 103, 107],
    'close': [105, 110, 115, 113, 117],
    'volume': [1000, 1100, 1200, 1150, 1250],
    'longenter': [0, 1, 0, 0, 0],
    'longexit': [0, 0, 1, 0, 0]
}
dates = pd.date_range(start="2022-01-01", periods=5)
ohlcv_df = pd.DataFrame(data, index=dates)

# Calculate daily returns
returns = ohlcv_df['close'].pct_change().dropna()

# Create a signal series with longenter and longexit
signal = pd.Series(0, index=ohlcv_df.index)
signal[ohlcv_df['longenter'] > 0] = 1  # Buy signal
signal[ohlcv_df['longexit'] > 0] = -1  # Sell signal

# Generate positions based on signals
position = signal.replace(0, np.nan).ffill().fillna(0)

# Ensure positions align with returns
position = position.loc[returns.index]

# Calculate strategy returns
strategy_returns = position.shift(1) * returns

# Remove NaNs from strategy returns
strategy_returns = strategy_returns.dropna()

# Generate tear sheet
qs.reports.html(strategy_returns, output='tearsheet3.html')


/usr/local/lib/python3.10/dist-packages/quantstats/_plotting/core.py:516: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  _sns.kdeplot(data=combined_returns, color="black", ax=ax)


In [ ]:
import math

def find_square_range(price):
    # Find the closest integer square root
    lower_sqrt = int(math.sqrt(price))
    upper_sqrt = lower_sqrt + 1

    # Calculate the squares of these integers
    lower_square = lower_sqrt ** 2
    upper_square = upper_sqrt ** 2

    # Check which range the price falls into
    if lower_square <= price < upper_square:
        return lower_sqrt, upper_sqrt
    else:
        return upper_sqrt, upper_sqrt + 1

# Example usage
price = 63
square_range = find_square_range(price)
print(f"The price {price} is in the square range of {square_range[0]} and {square_range[1]}")


The price 63 is in the square range of 7 and 8


In [ ]:
import math

def find_square_range(price):
    # Find the closest lower integer square root
    lower_sqrt = int(math.sqrt(price))

    # Initialize the lower and upper bounds of the range
    lower_bound = lower_sqrt
    upper_bound = lower_sqrt + 0.25

    # Iterate through the ranges in steps of 0.25
    while upper_bound**2 < price:
        lower_bound += 0.25
        upper_bound += 0.25

    return lower_bound, upper_bound

# Example usage
price = 297.75
square_range = find_square_range(price)
print(f"The price {price} is in the square range of {square_range[0]} and {square_range[1]}")


The price 297.75 is in the square range of 17.25 and 17.5


In [ ]:
import math

def calculate_angle2(N):
    theta = math.sqrt(N) * 180 #- 45
    theta_decimal = (theta % 360) / 360  # Get the decimal part of theta/360
    theta_final = theta_decimal * 360  # Convert back to degrees
    return theta_final

def calculate_number_on_radial(k, theta):
    N_k = (2 * k + 2 * (theta / 360) + 0.25) ** 2
    return N_k

def calculate_N(reference_N , reference_theta , theta_final):
    reference_N = reference_N
    reference_theta = reference_theta
    theta_decimal = theta_final / 360
    theta = theta_decimal * 360
    if theta == 0:
        N = reference_N
    else:
        theta_offset = theta - reference_theta
        sqrt_N = math.sqrt(reference_N)
        N = ((theta_offset / 180) + sqrt_N) ** 2
    return N

def calculate_N2(reference_N , reference_theta , theta_final):
    reference_N = reference_N
    reference_theta = reference_theta
    theta_decimal = theta_final / 360
    theta = theta_decimal * 360
    if theta == 0:
        N = reference_N
    else:
        theta_offset = theta - reference_theta
        sqrt_N = math.sqrt(reference_N)
        N = ((theta_offset / 180) - sqrt_N) ** 2
    return N

# N = 48842
# theta = calculate_angle2(N)
# print(f"Angle θ 2 for N={N}: {theta} degrees")
# k = 3
# N_k = calculate_number_on_radial(k, theta)
# print(f"Number N_k for k={k} and θ={theta}: {N_k}")

# angles = [11.25, 22.5, 33.75, 45, 56.25, 60, 67.5, 71.25, 75, 78.75, 82.5, 86.25, 90]
angles = [0.1,11.25, 22.5, 33.75,45,90,135,180,225,240,270,300,315,360]

# 148.482944
reference_N = 297.75
reference_theta = calculate_angle2(reference_N)
print("reference_N",reference_N,"reference_theta ",reference_theta)
long = []
for angle in angles:
    N = calculate_N(reference_N, reference_theta, angle)
    long.append(N)
    print("Angle: ",angle,"N ",N)
long.append(reference_N)
print()
short = []
for angle in angles:
    N = calculate_N2(reference_N, reference_theta, angle)
    print("Angle: ",angle,"N ",N)
    short.append(N)

long.sort(reverse=False)
short.sort(reverse=True)  # Sort the list in descending order
long,short

reference_N 297.75 reference_theta  225.9781068127313
Angle:  0.1 N  256.0177780864197
Angle:  11.25 N  258.00390625
Angle:  22.5 N  260.015625
Angle:  33.75 N  262.03515625
Angle:  45 N  264.0625
Angle:  90 N  272.25
Angle:  135 N  280.5625
Angle:  180 N  289.0
Angle:  225 N  297.5625
Angle:  240 N  300.4444444444444
Angle:  270 N  306.25
Angle:  300 N  312.11111111111114
Angle:  315 N  315.0625
Angle:  360 N  324.0

Angle:  0.1 N  342.6316613665002
Angle:  11.25 N  340.3422764571221
Angle:  22.5 N  338.0401367254378
Angle:  33.75 N  335.7458094937534
Angle:  45 N  333.4592947620691
Angle:  90 N  324.3913608353317
Angle:  135 N  315.4484269085943
Angle:  180 N  306.6304929818569
Angle:  225 N  297.9375590551195
Angle:  240 N  295.0676921906515
Angle:  270 N  289.3696251283821
Angle:  300 N  283.72711362166825
Angle:  315 N  280.9266912016447
Angle:  360 N  272.6087572749073


([256.0177780864197,
  258.00390625,
  260.015625,
  262.03515625,
  264.0625,
  272.25,
  280.5625,
  289.0,
  297.5625,
  297.75,
  300.4444444444444,
  306.25,
  312.11111111111114,
  315.0625,
  324.0],
 [342.6316613665002,
  340.3422764571221,
  338.0401367254378,
  335.7458094937534,
  333.4592947620691,
  324.3913608353317,
  315.4484269085943,
  306.6304929818569,
  297.9375590551195,
  295.0676921906515,
  289.3696251283821,
  283.72711362166825,
  280.9266912016447,
  272.6087572749073])

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_data(ticker, start, end):
    """Fetch OHLCV data from Yahoo Finance."""
    data = yf.download(ticker, start=start, end=end)
    return data

def is_bullish_engulfing(row):
    """Check if the row matches a bullish engulfing pattern."""
    return (
        row['Open'] < row['Close'] and
        row['Open'].shift(1) > row['Close'].shift(1) and
        row['Open'] < row['Close'].shift(1) and
        row['Close'] > row['Open'].shift(1)
    )

def is_bearish_engulfing(row):
    """Check if the row matches a bearish engulfing pattern."""
    return (
        row['Open'] > row['Close'] and
        row['Open'].shift(1) < row['Close'].shift(1) and
        row['Open'] > row['Close'].shift(1) and
        row['Close'] < row['Open'].shift(1)
    )

def apply_engulfing_patterns(data):
    """Apply engulfing pattern recognition."""
    data['Bullish_Engulfing'] = data.apply(is_bullish_engulfing, axis=1)
    data['Bearish_Engulfing'] = data.apply(is_bearish_engulfing, axis=1)
    return data

if __name__ == "__main__":
    # Example usage
    ticker = "AAPL"  # You can replace this with any ticker symbol
    start_date = "2023-01-01"
    end_date = "2024-01-01"

    data = fetch_data(ticker, start=start_date, end=end_date)
    data = apply_engulfing_patterns(data)

    # Display rows where engulfing patterns are found
    bullish_engulfing = data[data['Bullish_Engulfing']]
    bearish_engulfing = data[data['Bearish_Engulfing']]

    print("Bullish Engulfing Patterns:")
    print(bullish_engulfing)

    print("\nBearish Engulfing Patterns:")
    print(bearish_engulfing)

[*********************100%%**********************]  1 of 1 completed


AttributeError: 'numpy.float64' object has no attribute 'shift'

In [ ]:
import yfinance as yf
import pandas as pd

# Fetch data
ticker = 'ETC-USD'  # Example ticker
data = yf.download(ticker, start='2023-01-01', end='2023-12-31')
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-01,15.694401,15.822981,15.615783,15.772024,15.772024,56090451
2023-01-02,15.773082,16.230122,15.540663,16.112904,16.112904,89970297
2023-01-03,16.113806,16.165335,15.672698,15.893769,15.893769,88486338
2023-01-04,15.893930,19.603304,15.866254,19.186888,19.186888,514665601
2023-01-05,19.185896,19.395088,18.127031,18.215586,18.215586,355882822


In [ ]:
def is_bullish_engulfing(current, previous):
    return (current['Close'] > current['Open']) and (previous['Close'] < previous['Open']) and \
           (current['Open'] < previous['Close']) and (current['Close'] > previous['Open'])

def is_bearish_engulfing(current, previous):
    return (current['Close'] < current['Open']) and (previous['Close'] > previous['Open']) and \
           (current['Open'] > previous['Close']) and (current['Close'] < previous['Open'])

In [ ]:
# Initialize columns for patterns
data['Bullish_Engulfing'] = False
data['Bearish_Engulfing'] = False

# Iterate through the data to find patterns
for i in range(1, len(data)):
    current = data.iloc[i]
    previous = data.iloc[i-1]

    if is_bullish_engulfing(current, previous):
        data.at[data.index[i], 'Bullish_Engulfing'] = True

    if is_bearish_engulfing(current, previous):
        data.at[data.index[i], 'Bearish_Engulfing'] = True

# Filter the data to see where the patterns occur
bullish_engulfing_data = data[data['Bullish_Engulfing']]
bearish_engulfing_data = data[data['Bearish_Engulfing']]

print("Bullish Engulfing Patterns:")
print(bullish_engulfing_data)

print("Bearish Engulfing Patterns:")
print(bearish_engulfing_data)

data.head()


Bullish Engulfing Patterns:
                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2023-01-06  18.215511  20.675533  17.941557  20.440617  20.440617  381873531   
2023-01-13  20.917381  21.827747  20.692137  21.610418  21.610418  320452820   
2023-01-29  21.725510  22.792833  21.618933  22.774588  22.774588  258986708   
2023-02-07  22.141218  23.265928  22.124727  23.200802  23.200802  221619492   
2023-02-14  20.672995  21.236603  20.414509  21.220442  21.220442  162916457   
2023-02-26  20.989780  21.542967  20.857466  21.415094  21.415094  114746392   
2023-03-12  17.299337  18.833534  17.052736  18.808155  18.808155  149812179   
2023-04-22  19.333012  20.449976  19.330244  20.210840  20.210840  175109135   
2023-04-25  19.422998  20.006348  19.122068  19.800358  19.800358  109126833   
2023-06-04  18.136354  18.390251  18.102739  18.218000  18.218000   67454282   
2023-06-23  

,Open,High,Low,Close,Adj Close,Volume,Bullish_Engulfing,Bearish_Engulfing
Date,,,,,,,,
2023-01-01,15.694401,15.822981,15.615783,15.772024,15.772024,56090451,False,False
2023-01-02,15.773082,16.230122,15.540663,16.112904,16.112904,89970297,False,False
2023-01-03,16.113806,16.165335,15.672698,15.893769,15.893769,88486338,False,False
2023-01-04,15.893930,19.603304,15.866254,19.186888,19.186888,514665601,False,False
2023-01-05,19.185896,19.395088,18.127031,18.215586,18.215586,355882822,False,False


In [ ]:
# !pip install mplfinance

In [ ]:
import mplfinance as mpf
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

def fetch_data(ticker, start, end, interval):
    return yf.download(ticker, start=start, end=end, interval=interval)

# Function to identify engulfing patterns
def identify_engulfing_patterns(data):
    data['BullishEngulfing'] = False
    data['BearishEngulfing'] = False
    data['HighestHigh'] = np.nan
    data['LowestLow'] = np.nan

    for i in range(1, len(data)):
        prev_open = data['Open'][i-1]
        prev_close = data['Close'][i-1]
        prev_high = data['High'][i-1]
        prev_low = data['Low'][i-1]

        curr_open = data['Open'][i]
        curr_close = data['Close'][i]
        curr_high = data['High'][i]
        curr_low = data['Low'][i]

        # Bullish Engulfing Pattern
        if prev_close < prev_open and curr_close > curr_open and curr_close > prev_open and curr_open < prev_close:
            data.at[i, 'BullishEngulfing'] = True
            data.at[i, 'HighestHigh'] = max(prev_high, curr_high)
            data.at[i, 'LowestLow'] = min(prev_low, curr_low)

        # Bearish Engulfing Pattern
        if prev_close > prev_open and curr_close < curr_open and curr_open > prev_close and curr_close < prev_open:
            data.at[i, 'BearishEngulfing'] = True
            data.at[i, 'HighestHigh'] = max(prev_high, curr_high)
            data.at[i, 'LowestLow'] = min(prev_low, curr_low)

    return data

# Fetch data
ticker = 'ETC-USD'
start_date = '2024-06-01'
end_date = '2024-06-10'
interval = '5m'
data = fetch_data(ticker, start_date, end_date, interval)
data.reset_index(inplace=True, drop=False)
data['color'] = np.where(data['Close'] > data['Open'], 'green', 'red')
data.head()

[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,color
0,2024-06-01 00:00:00+00:00,29.613632,29.613632,29.575727,29.575727,29.575727,0,red
1,2024-06-01 00:05:00+00:00,29.577576,29.590609,29.577576,29.584881,29.584881,1057712,green
2,2024-06-01 00:10:00+00:00,29.569593,29.570807,29.561144,29.561144,29.561144,0,red
3,2024-06-01 00:15:00+00:00,29.556313,29.556313,29.517166,29.517166,29.517166,43968,red
4,2024-06-01 00:20:00+00:00,29.516493,29.516493,29.480408,29.480408,29.480408,317904,red


In [ ]:

# Identify patterns
data = identify_engulfing_patterns(data)
# Initialize Inside Candle columns
data['InsideCandle'] = False

# Initialize Inside Candle columns
data['InsideCandle'] = False
data['MotherCandleHigh'] = pd.NA
data['MotherCandleLow'] = pd.NA

# Identify Inside Candle patterns and record Mother Candle High and Low
for i in range(1, len(data)):
    if (data['High'][i] < data['High'][i-1]) and (data['Low'][i] > data['Low'][i-1]):
      if (data['color'][i] != data['color'][i-1] ):
        data.at[i, 'InsideCandle'] = True
        data.at[i, 'MotherCandleHigh'] = data['High'][i-1]
        data.at[i, 'MotherCandleLow'] = data['Low'][i-1]
data.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,color,BullishEngulfing,BearishEngulfing,HighestHigh,LowestLow,InsideCandle,MotherCandleHigh,MotherCandleLow
0,2024-06-01 00:00:00+00:00,29.613632,29.613632,29.575727,29.575727,29.575727,0,red,False,False,NaN,NaN,False,<NA>,<NA>
1,2024-06-01 00:05:00+00:00,29.577576,29.590609,29.577576,29.584881,29.584881,1057712,green,False,False,NaN,NaN,True,29.613632,29.575727
2,2024-06-01 00:10:00+00:00,29.569593,29.570807,29.561144,29.561144,29.561144,0,red,False,False,NaN,NaN,False,<NA>,<NA>
3,2024-06-01 00:15:00+00:00,29.556313,29.556313,29.517166,29.517166,29.517166,43968,red,False,False,NaN,NaN,False,<NA>,<NA>
4,2024-06-01 00:20:00+00:00,29.516493,29.516493,29.480408,29.480408,29.480408,317904,red,False,False,NaN,NaN,False,<NA>,<NA>


In [ ]:
data[data['InsideCandle']==True]

,Datetime,Open,High,Low,Close,Adj Close,Volume,color,BullishEngulfing,BearishEngulfing,HighestHigh,LowestLow,InsideCandle,MotherCandleHigh,MotherCandleLow
1,2024-06-01 00:05:00+00:00,29.577576,29.590609,29.577576,29.584881,29.584881,1057712,green,False,False,NaN,NaN,True,29.613632,29.575727
7,2024-06-01 00:35:00+00:00,29.501553,29.501553,29.485233,29.486876,29.486876,0,red,False,False,NaN,NaN,True,29.50173,29.467806
28,2024-06-01 02:20:00+00:00,29.575432,29.577198,29.568747,29.568747,29.568747,0,red,False,False,NaN,NaN,True,29.577736,29.566118
31,2024-06-01 02:35:00+00:00,29.626167,29.626167,29.601734,29.603844,29.603844,318864,red,False,False,NaN,NaN,True,29.638426,29.57115
32,2024-06-01 02:40:00+00:00,29.605165,29.616392,29.605165,29.616392,29.616392,4256,green,False,False,NaN,NaN,True,29.626167,29.601734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2024-06-09 20:40:00+00:00,26.882622,26.886465,26.882622,26.886456,26.886456,31280,green,False,False,NaN,NaN,True,26.891956,26.872742
2554,2024-06-09 20:50:00+00:00,26.873400,26.884960,26.873400,26.884960,26.884960,62256,green,False,False,NaN,NaN,True,26.886379,26.872866
2572,2024-06-09 22:20:00+00:00,26.976156,26.976156,26.958464,26.959232,26.959232,152512,red,False,False,NaN,NaN,True,26.976513,26.954288
2584,2024-06-09 23:20:00+00:00,26.949327,26.955187,26.949327,26.955187,26.955187,0,green,False,False,NaN,NaN,True,26.9603,26.948748


In [ ]:

import plotly.graph_objects as go
# Plotting the data with Plotly
fig = go.Figure()

# Add candlestick trace
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name='Candlesticks'))

# Add scatter trace for Inside Candles
fig.add_trace(go.Scatter(
    x=data.index[data['InsideCandle']],
    y=data['Close'][data['InsideCandle']],
    mode='markers',
    name='Inside Candle',
    marker=dict(color='blue', size=10, symbol='diamond'),
))

# Add scatter trace for Mother Candle Highs
fig.add_trace(go.Scatter(
    x=data.index[data['InsideCandle']],
    y=data['MotherCandleHigh'][data['InsideCandle']],
    mode='markers',
    name='Mother Candle High',
    marker=dict(color='olive', size=10, symbol='triangle-up'),
))

# Add scatter trace for Mother Candle Lows
fig.add_trace(go.Scatter(
    x=data.index[data['InsideCandle']],
    y=data['MotherCandleLow'][data['InsideCandle']],
    mode='markers',
    name='Mother Candle Low',
    marker=dict(color='crimson', size=10, symbol='triangle-down'),
))

fig.add_trace(go.Scatter(
    x=data.index[data['BullishEngulfing']],
    y=data['Close'][data['BullishEngulfing']],
    mode='markers',
    name='Bullish Engulfing Close',
    marker=dict(color='green', size=10, symbol='triangle-up'),
))

# Add scatter trace for Bearish Engulfing close prices
fig.add_trace(go.Scatter(
    x=data.index[data['BearishEngulfing']],
    y=data['Close'][data['BearishEngulfing']],
    mode='markers',
    name='Bearish Engulfing Close',
    marker=dict(color='red', size=10, symbol='triangle-down'),
))

# Update layout
fig.update_layout(   title=ticker + ' Candlestick Chart with Engulfing Patterns',    yaxis_title='Price',    xaxis_title='Date',    xaxis_rangeslider_visible=False,   height=800,    showlegend=True )

fig.show()

In [ ]:
import math

def calculate_N2(reference_N , reference_theta , theta_final):
    reference_N = reference_N
    reference_theta = reference_theta
    theta_decimal = theta_final / 360
    theta = theta_decimal * 360
    if theta == 0:
        N = reference_N
    else:
        theta_offset = theta - reference_theta
        sqrt_N = math.sqrt(reference_N)
        N = ((theta_offset / 180) - sqrt_N) ** 2
    return N

def calculate_angle2(N):
    theta = math.sqrt(N) * 180 #- 45
    theta_decimal = (theta % 360) / 360  # Get the decimal part of theta/360
    theta_final = theta_decimal * 360  # Convert back to degrees
    return theta_final

def calculate_N(reference_N , reference_theta , theta_final):
    reference_N = reference_N
    reference_theta = reference_theta
    theta_decimal = theta_final / 360
    theta = theta_decimal * 360
    if theta == 0:
        N = reference_N
    else:
        theta_offset = theta - reference_theta
        sqrt_N = math.sqrt(reference_N)
        N = ((theta_offset / 180) + sqrt_N) ** 2
    return N


long = []
short = []
angles = [0.1,11.25, 22.5, 33.75,45,90,135,180,225,270,315,360]

reference_N = 1466
# reference_N = data['Close'][i]
reference_theta = calculate_angle2(reference_N)
for angle in angles:
    N = calculate_N(reference_N, reference_theta, angle)
    long.append(N)
for angle in angles:
    N = calculate_N2(reference_N, reference_theta, angle)
    short.append(N)
long.sort(reverse=False)
short.sort(reverse=True)  # Sort the list in descending order

In [ ]:
long

[1444.0422225308644,
 1448.75390625,
 1453.515625,
 1458.28515625,
 1463.0625,
 1482.25,
 1501.5625,
 1521.0,
 1540.5625,
 1560.25,
 1580.0625,
 1600.0]

In [ ]:
short

[1488.1234626439225,
 1483.3481367911606,
 1478.537760681622,
 1473.7351970720838,
 1468.9404459625455,
 1449.8395665243922,
 1430.863687086239,
 1412.0128076480858,
 1393.2869282099323,
 1374.686048771779,
 1356.2101693336258,
 1337.8592898954726]

In [5]:
import math

N = 17.739
angle = math.sqrt(N) * 180 - 45
angle = angle /360
decimal_part = angle % 1 * 100
req_angle = decimal_part*360
req_angle

35311.84604004838

In [11]:
def calculate_angle2(N):
    angle = math.sqrt(N) * 180 - 45
    angle = angle /360
    decimal_part = angle % 1 * 100
    theta_final = decimal_part*360
    return theta_final

def calculate_N( N, k):
    theta = calculate_angle2(N)
    N = (2 * k + 2 * theta / 360 + 0.25) ** 2
    return N

k = 1
N = 17.739 # price
N = calculate_N(N, k)
print(f"The value of N for θ = {req_angle} and k = {k} is {N}.")

The value of N for θ = 35311.84604004838 and k = 1 is 39373.24355088024.


In [10]:

angle = 176.45
decimal_part = angle % 1 #* 100
decimal_part

0.44999999999998863